<a href="https://colab.research.google.com/github/rushikeshnakhate/PortfoliOpt/blob/master/multiple_dates_expected_return_df_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from pathlib import Path
import pandas as pd
from tabulate import tabulate
data_dir = Path(r"D:\PortfoliOpt\data")
# df1 = pd.read_pickle(r"D:\PortfoliOpt\data\202001\stocks.pkl")
# df1 = pd.read_pickle(r"D:\PortfoliOpt\data\202001\expected_return_all_type.pkl")
performance_df = pd.read_pickle(r"D:\PortfoliOpt\data\2019\performance_metrics.pkl")
print(tabulate(performance_df.head(10), headers='keys', tablefmt='pretty'))

In [18]:
post_processing_weight_df =  pd.read_pickle(r"D:\PortfoliOpt\data\2022_2025\performance_metrics.pkl")
#  write to xls

post_processing_weight_df.to_excel(r"D:\PortfoliOpt\data\2024\2022_2025.xlsx", index=False)

In [15]:
post_processing_weight_df.columns

Index(['Expected Annual Return', 'Annual Volatility', 'Sharpe Ratio',
       'Weights', 'Expected Return Type', 'Risk Model', 'Optimizer',
       'Allocation_CustomClusteredAllocator_weight',
       'Allocation_CustomClusteredAllocator_remaining_amount',
       'Allocation_CustomDiversityAllocator_weight',
       'Allocation_CustomDiversityAllocator_remaining_amount',
       'Allocation_CustomProportionalRoundingAllocator_weight',
       'Allocation_CustomProportionalRoundingAllocator_remaining_amount',
       'Allocation_CustomTransactionCostAllocator_weight',
       'Allocation_CustomTransactionCostAllocator_remaining_amount',
       'Allocation_CustomWeightedFloorAllocator_weight',
       'Allocation_CustomWeightedFloorAllocator_remaining_amount',
       'Allocation_GreedyPortfolio_weight',
       'Allocation_GreedyPortfolio_remaining_amount',
       'Allocation_LpPortfolio_weight',
       'Allocation_LpPortfolio_remaining_amount',
       'CustomClusteredAllocator_Volatility',
     

In [ ]:
def get_allocators_list(df):
    # Convert all relevant columns to numeric
    df = df.apply(pd.to_numeric, errors='coerce')
    # Identify allocator columns based on patterns in their names
    allocators = [
        col.split('_')[1]
        for col in df.columns
        if col.startswith("Allocation_")
    ]
    # Ensure unique allocators
    return list(set(allocators))


d = get_allocators_list(performance_df)
print(d)


In [102]:
import pandas as pd


def select_top_10_portfolios(df, top_n=10):
    """
    Selects the top portfolios based on Expected Annual Return, Annual Volatility, and Sharpe Ratio.
    Args:
    - df: DataFrame containing portfolio performance data with the following columns:
        - 'Expected Annual Return'
        - 'Annual Volatility'
        - 'Sharpe Ratio'
        - 'Weights'
    - top_n: Number of top portfolios to select based on combined criteria.
    
    Returns:
    - top_10_df: DataFrame containing the top 10 portfolios based on the selected criteria.
    """

    # Step 1: Normalize the data (to bring all metrics to a comparable scale)
    df_normalized = df.copy()
    df_normalized['Expected Annual Return'] = (df['Expected Annual Return'] - df['Expected Annual Return'].min()) / (
            df['Expected Annual Return'].max() - df['Expected Annual Return'].min())
    df_normalized['Annual Volatility'] = (df['Annual Volatility'].max() - df['Annual Volatility']) / (
            df['Annual Volatility'].max() - df['Annual Volatility'].min())
    df_normalized['Sharpe Ratio'] = (df['Sharpe Ratio'] - df['Sharpe Ratio'].min()) / (
            df['Sharpe Ratio'].max() - df['Sharpe Ratio'].min())

    # Step 2: Calculate a score for each portfolio
    # Higher Expected Annual Return and Sharpe Ratio are better, while lower Volatility is better.
    df_normalized['Score'] = (df_normalized['Expected Annual Return'] + df_normalized['Sharpe Ratio']) - df_normalized[
        'Annual Volatility']

    # Step 3: Sort the portfolios by score in descending order
    df_sorted = df_normalized.sort_values(by='Score', ascending=False)

    # Step 4: Select the top_n portfolios
    top_10_df = df_sorted.head(top_n)

    return top_10_df


# Example usage with your dataframe `df`
top_10_portfolios = select_top_10_portfolios(performance_df, 5)
print(tabulate(top_10_portfolios, headers='keys', tablefmt='pretty'))


+-----+------------------------+----------------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+-------------------------------+-------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------

In [100]:
import pandas as pd

def select_top_portfolios_for_allocator(df, allocator_name, top_n=10):
    """
    Selects the top portfolios for a given allocator based on Expected Annual Return, 
    Annual Volatility, and Sharpe Ratio.
    
    Args:
    - df: DataFrame containing portfolio performance data with the following columns:
        - 'Expected Annual Return'
        - 'Annual Volatility'
        - 'Sharpe Ratio'
        - 'Weights'
        - Columns for each allocator's performance (e.g., 'Allocator_Volatility', 'Allocator_Return', 'Allocator_Sharpe')
    - allocator_name: The name of the allocator to select top portfolios for.
    - top_n: Number of top portfolios to select based on combined criteria.
    
    Returns:
    - top_10_df: DataFrame containing the top 10 portfolios based on the selected allocator's performance criteria.
    """
    
    # Step 1: Filter the relevant columns for the given allocator
    expected_return_col = f'{allocator_name}_Return'
    volatility_col = f'{allocator_name}_Volatility'
    sharpe_ratio_col = f'{allocator_name}_Sharpe'
    weight_col = f'Allocation_{allocator_name}_weight'
    
    if expected_return_col not in df.columns or volatility_col not in df.columns or sharpe_ratio_col not in df.columns:
        raise ValueError(f"Allocator columns for {allocator_name} not found in DataFrame.")
    
    # Step 2: Normalize the data (to bring all metrics to a comparable scale)
    df_normalized = df.copy()
    
    # Normalize Expected Annual Return (higher is better)
    df_normalized[expected_return_col] = (df[expected_return_col] - df[expected_return_col].min()) / (df[expected_return_col].max() - df[expected_return_col].min())
    
    # Normalize Volatility (lower is better)
    df_normalized[volatility_col] = (df[volatility_col].max() - df[volatility_col]) / (df[volatility_col].max() - df[volatility_col].min())
    
    # Normalize Sharpe Ratio (higher is better)
    df_normalized[sharpe_ratio_col] = (df[sharpe_ratio_col] - df[sharpe_ratio_col].min()) / (df[sharpe_ratio_col].max() - df[sharpe_ratio_col].min())
    
    # Step 3: Calculate a score for each portfolio
    df_normalized['Score'] = (df_normalized[expected_return_col] + df_normalized[sharpe_ratio_col]) - df_normalized[volatility_col]
    
    # Step 4: Sort the portfolios by score in descending order
    df_sorted = df_normalized.sort_values(by='Score', ascending=False)
    
    # Step 5: Select the top_n portfolios
    top_10_df = df_sorted.head(top_n)
    
    return top_10_df





In [101]:
# List of allocators to process
allocators = [
    'CustomTransactionCostAllocator', 'LpPortfolio', 
    'CustomWeightedFloorAllocator', 'CustomDiversityAllocator', 
    'GreedyPortfolio', 'CustomClusteredAllocator', 
    'CustomProportionalRoundingAllocator'
]

# Loop through the list of allocators
top_portfolios_dict = {}
for allocator in allocators:
    top_10_portfolios = select_top_portfolios_for_allocator(performance_df, allocator)
    top_portfolios_dict[allocator] = top_10_portfolios
    
    

+------+------------------------+----------------------+--------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------+------------------------+-------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------

In [110]:
print(tabulate(performance_df.head(10), headers='keys', tablefmt='pretty'))

+---+------------------------+----------------------+---------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------+-----------------------------------+-------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------+------------